In [15]:
import tensorflow as tf
import numpy as np
from numpy import ndarray
import matplotlib.pyplot as plt
import time
import cv2
import os
import requests
import IPython.display as display
import PIL.Image
from tqdm import tqdm

from cv2 import cvtColor
from cv2 import imread
from cv2 import COLOR_BGR2RGB

from keras import Model
from keras.preprocessing.image import load_img, img_to_array
from os import listdir
from keras import Sequential

from data import Data_Augmentation
from model import Create_transfert_learning_Model
from constant import IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_COLOR

from augmentator import RandomColorShifting

from keras.layers import Rescaling, Resizing

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from keras.layers import Layer


In [16]:
class Data_Processing:
    
    def __init__(self, IMAGE_HEIGHT=224, IMAGE_WIDTH=224, nbr_augmentation=10, train_size=0.90, dev_size=0.30):
        self.height = IMAGE_HEIGHT
        self.width = IMAGE_WIDTH
        self.nbr_augmentation = nbr_augmentation
        self.train_size = train_size
        self.dev_size = dev_size
        self.train_set = list()
        self.dev_set = list()
        self.test_set = list()
        self.Resizer = Resizing(224, 224)
        self.Rescaler = Rescaling(1./255)
        self.Augmentator = Data_Augmentation(nbr_augmentation)

    def load_img(self, path : str):
        
        img = load_img(path)
        img = self.Resizer(img)
        img = img_to_array(img)
        img = self.Rescaler(img)
        return (img)

    def split_data_in_set(self, set : tuple):
        
        train_random = np.random.uniform(0, 1)
        dev_random = np.random.uniform(0, 1)
        
        if (train_random < self.train_size):
            self.train_set.append(set)
        else :
            if (dev_random < self.dev_size):
                self.dev_set.append(set)
            else :
                self.test_set.append(set)
        
        
    def labeling_img_directory(self, directory : str, classe : int) :

        for filename in tqdm(listdir(directory)):

            path = directory + '/' + filename
            img = self.load_img(path)
            augmented_imgs = self.Augmentator(img)
            for augmented in augmented_imgs:
                self.split_data_in_set((augmented, classe))

    def create_X_Y(self, list_tuple : list):
    
        X = list()
        Y = list()
        
        for tuple in list_tuple :
            X.append(tuple[0])
            Y.append(tuple[1])

        X = np.array(X)
        Y = np.array(Y)
        Y = Y.reshape(Y.shape[0], 1)
    
        return (X, Y)

    def output(self, healthy_dir_name : str,  disease_dir_name : str):
        
        self.labeling_img_directory(directory=disease_dir_name, classe=1)
        self.labeling_img_directory(directory=healthy_dir_name, classe=0)
        X_train, Y_train = self.create_X_Y(self.train_set)
        X_dev, Y_dev = self.create_X_Y(self.dev_set)
        X_test, Y_test = self.create_X_Y(self.test_set)
        
        print(X_train.shape, Y_train.shape)
        print(X_dev.shape, Y_dev.shape)
        print(X_test.shape, Y_test.shape)
        
        return (X_train, Y_train,
                X_dev, Y_dev,
                X_test, Y_test)

In [17]:
Processor = Data_Processing()

In [18]:
X_train, Y_train,\
X_dev, Y_dev,\
X_test, Y_test = Processor.output(healthy_dir_name='leaves_img/sain',
                                  disease_dir_name='leaves_img/malade')

100%|██████████| 118/118 [00:13<00:00,  8.73it/s]


In [ ]:
model = Create_transfert_learning_Model(2)
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   